<a href="https://colab.research.google.com/github/Hongyoosung/Research_Paper_Writing_Agent/blob/main/research_paper_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Complete Research Paper Writing Agent using OpenAI API (gpt-4o-mini)
"""

# Cell 1: 필요한 라이브러리 설치 및 구글 드라이브 마운트
!pip install -q openai chromadb arxiv requests torch sentence-transformers langdetect scikit-learn rouge
!apt-get update -q
!apt-get install -y -q texlive texlive-latex-extra texlive-fonts-recommended texlive-science texlive-publishers

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Cell 2: 필요한 모듈 임포트
import os
import datetime
import json
import time
import requests
import re
import numpy as np
import zipfile
import arxiv
import chromadb
from google.colab import files
from google.colab import userdata
from langdetect import detect
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
# Cell 3: 연구 주제 설정
RESEARCH_TOPIC = "minimizing prompt bloat in single-agent systems using Language Models with GOAP and decision trees"
KEYWORDS = "prompt bloat prompt optimization language model single agent"
TITLE_PROMPT = f"""You are the most intelligent and innovative AI, capable of producing groundbreaking and feasible research. Generate a compelling and specific academic paper title for research on {RESEARCH_TOPIC}. The title should be concise (10-15 words), informative, and highlight efficiency improvements in agent performance. Provide only the title without any additional explanation. Ensure 'LLM' refers to 'Language Models'. Do not use Markdown formatting."""

In [ ]:
# Cell 4: 논문 길이 설정
SECTION_LENGTHS = {
    'abstract': 300,    # 약 250-400 단어
    'introduction': 600, # 약 500-700 단어
    'related_work': 700, # 약 600-800 단어
    'methodology': 800,  # 약 700-900 단어
    'experiments': 700,  # 약 600-850 단어
    'conclusion': 400    # 약 300-400 단어
}

In [ ]:
# Cell 5: 작업 디렉토리 설정
folder_name = datetime.datetime.now().strftime("research_paper_%Y%m%d_%H%M%S")
output_dir = f"/content/drive/MyDrive/Research_Papers/{folder_name}"
os.makedirs(output_dir, exist_ok=True)
os.chdir(output_dir)
print(f"작업 디렉토리: {output_dir}")

In [ ]:
# Cell 6: 로깅 유틸리티 설정
class ResearchLogger:
    def __init__(self, output_dir):
        self.log_file = os.path.join(output_dir, 'research_log.txt')
        self.start_time = time.time()
        self.output_dir = output_dir

    def log(self, message, level='INFO'):
        current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        elapsed_time = time.time() - self.start_time
        hours = int(elapsed_time // 3600)
        minutes = int((elapsed_time % 3600) // 60)
        seconds = int(elapsed_time % 60)

        log_message = f"[{current_time}] [{level}] [Elapsed: {hours:02d}:{minutes:02d}:{seconds:02d}] {message}"

        print(log_message)
        with open(self.log_file, 'a', encoding='utf-8') as f:
            f.write(log_message + '\n')

logger = ResearchLogger(output_dir)
logger.log(f"연구 논문 작성 에이전트 시작 - 작업 디렉토리: {output_dir}", "START")

In [ ]:
# Cell 7: LaTeX PDF 생성 테스트
logger.log("LaTeX PDF 생성 테스트 시작")

test_latex = r"""
\documentclass{article}
\usepackage[utf8]{inputenc}
\begin{document}
\title{LaTeX Test Document}
\author{Test Author}
\date{\today}
\maketitle
This is a test document to verify LaTeX installation.
\section{Test Section}
Testing LaTeX compilation with UTF-8 encoding.
\end{document}
"""

test_tex_path = os.path.join(output_dir, 'test.tex')
test_pdf_path = os.path.join(output_dir, 'test.pdf')

with open(test_tex_path, 'w', encoding='utf-8') as f:
    f.write(test_latex)

result = os.system(f'cd {output_dir} && pdflatex -interaction=nonstopmode test.tex > test_compile.log 2>&1')

if os.path.exists(test_pdf_path):
    logger.log("LaTeX 테스트 성공: PDF 생성 확인")
else:
    logger.log("LaTeX 테스트 실패: PDF 생성 불가", "ERROR")
    with open(os.path.join(output_dir, 'test_compile.log'), 'r') as f:
        logger.log(f"컴파일 로그:\n{f.read()}", "ERROR")
    raise Exception("LaTeX setup failed")

In [ ]:
# Cell 8: OpenAI API 클라이언트 설정
logger.log("OpenAI API 클라이언트 초기화 시작")
try:
    api_key = userdata.get('OPENAI')
    if not api_key:
        raise ValueError("OPENAI 비밀키가 설정되지 않았습니다.")
    client = OpenAI(api_key=api_key)
    logger.log("OpenAI API 클라이언트 초기화 완료")
except Exception as e:
    logger.log(f"OpenAI API 초기화 실패: {str(e)}", "ERROR")
    raise

In [ ]:
# Cell 9: Chroma DB를 위한 임베딩 모델 설정
logger.log("임베딩 모델 로딩 시작")
embedder = SentenceTransformer('all-MiniLM-L6-v2')
logger.log("임베딩 모델 로딩 완료")

In [ ]:
# Cell 10: Chroma DB 초기화
logger.log("Chroma DB 초기화 시작")
chroma_client = chromadb.Client()

try:
    collection = chroma_client.create_collection(
        name="research_papers",
        get_or_create=True
    )
    logger.log("Chroma DB 초기화 완료")
except Exception as e:
    logger.log(f"Chroma DB 초기화 실패: {str(e)}", "ERROR")
    try:
        chroma_client.delete_collection("research_papers")
        logger.log("기존 research_papers 컬렉션 삭제")
        collection = chroma_client.create_collection("research_papers")
        logger.log("새 research_papers 컬렉션 생성 완료")
    except Exception as e2:
        logger.log(f"컬렉션 재생성 실패: {str(e2)}", "ERROR")
        raise

In [ ]:
# Cell 11: 텍스트 생성 함수 - OpenAI API (gpt-4o-mini) 사용
def generate_text(prompt, max_length=800, temperature=0.7, max_attempts=3):
    best_text = ""
    best_score = -1
    model = "gpt-4o-mini"

    for attempt in range(max_attempts):
        try:
            messages = [{"role": "user", "content": prompt}]
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=max_length,
                temperature=temperature + (attempt * 0.1),
                top_p=0.9
            )

            generated_text = response.choices[0].message.content.strip()

            # 프롬프트 관련 텍스트 제거 (인용 패턴은 유지)
            prompt_patterns = [
                r"Write a.*?\n",
                r"Include:.*?\n",
                r"1\..*?\n",
                r"Ensure '.*?\n",
                r"Do not use Markdown.*?\n",
                r"You are the most.*?\n"
            ]
            for pattern in prompt_patterns:
                try:
                    generated_text = re.sub(pattern, "", generated_text, flags=re.DOTALL)
                except Exception as e:
                    logger.log(f"프롬프트 제거 오류: {str(e)}", "WARNING")
                    continue

            logger.log(f"생성된 텍스트 시작 (시도 {attempt+1}): {generated_text[:100]}...")

            word_count = len(generated_text.split())
            score = word_count / max_length if word_count > 50 else 0

            logger.log(f"시도 {attempt+1}/{max_attempts}: {word_count} words, 점수: {score:.2f}")

            if score > best_score:
                best_score = score
                best_text = generated_text

            if best_score > 0.9:
                break

        except Exception as e:
            logger.log(f"텍스트 생성 오류 (시도 {attempt+1}): {str(e)}", "WARNING")
            time.sleep(2)
            continue

    if best_text:
        logger.log(f"최종 텍스트 선택: {len(best_text.split())} words, 점수: {best_score:.2f}")
        return best_text
    else:
        logger.log("텍스트 생성 실패: 모든 시도에서 유효한 결과 없음", "ERROR")
        return "Error generating text"

In [ ]:
# Cell 12: ArXiv에서 논문 수집 함수
def fetch_papers(keywords, num_papers=10):
    logger.log(f"ArXiv에서 논문 검색 시작: {keywords}")

    session = requests.Session()
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('http://', HTTPAdapter(max_retries=retries))
    session.mount('https://', HTTPAdapter(max_retries=retries))

    client = arxiv.Client()
    search = arxiv.Search(
        query=keywords + " AND (language model OR prompt optimization OR prompt compression)",
        max_results=num_papers * 2,
        sort_by=arxiv.SortCriterion.Relevance
    )

    try:
        papers = list(client.results(search))
        logger.log(f"원본 논문 수: {len(papers)}")

        filtered_papers = [p for p in papers if any(term in p.summary.lower() for term in ["language model", "prompt", "agent"])]
        logger.log(f"필터링 후 논문 수: {len(filtered_papers)}")

        bibtex_entries = []
        seen_titles = set()

        for i, paper in enumerate(filtered_papers[:num_papers]):
            try:
                if paper.title in seen_titles:
                    continue
                seen_titles.add(paper.title)
                arxiv_id = paper.entry_id.split('/')[-1]

                # Create a backup bibtex entry
                backup_bibtex = f"""@article{{paper{i},
  author = {{{', '.join([author.name for author in paper.authors])}}},
  title = {{{paper.title}}},
  journal = {{arXiv preprint arXiv:{arxiv_id}}},
  year = {{{paper.published.year}}},
  url = {{{paper.entry_id}}},
}}"""

                bibtex_url = f"http://arxiv.org/bibtex/{arxiv_id}"
                response = session.get(bibtex_url, timeout=10)
                if response.status_code == 200:
                    bibtex_entry = response.text.strip()
                    bibtex_entry = re.sub(r'@article{[^,]+,', f'@article{{paper{i},', bibtex_entry)
                    if not bibtex_entry.strip().endswith("}"):
                        bibtex_entry = bibtex_entry.strip() + "\n}"
                    bibtex_entries.append(bibtex_entry)
                    logger.log(f"논문 {i+1} BibTeX 수집 완료: {paper.title[:50]}...")
                else:
                    logger.log(f"논문 {i+1} BibTeT 요청 실패: 상태 코드 {response.status_code}. 백업 항목 사용", "WARNING")
                    bibtex_entries.append(backup_bibtex)
            except Exception as e:
                logger.log(f"논문 {i+1} BibTeX 수집 실패: {str(e)}. 백업 항목 사용", "WARNING")
                try:
                    bibtex_entries.append(backup_bibtex)
                except Exception as e2:
                    logger.log(f"백업 BibTeX 생성 실패: {str(e2)}", "ERROR")

        logger.log(f"수집된 논문: {len(filtered_papers)}개, 고유 BibTeX 항목: {len(bibtex_entries)}개")
        return filtered_papers[:num_papers], bibtex_entries

    except Exception as e:
        logger.log(f"ArXiv 검색 실패: {str(e)}", "ERROR")
        return [], []

In [ ]:
# Cell 13: RAG 쿼리 함수
def rag_query(query, n_results=3):
    query_vector = embedder.encode(query).tolist()
    results = collection.query(
        query_embeddings=[query_vector],
        n_results=n_results
    )
    return results['documents'][0] if results['documents'] else [], results['metadatas'][0] if results['metadatas'] else []

In [ ]:
# Cell 14: 품질 평가 함수
def calculate_quality_score(section_text, section_type):
    scores = {}
    word_count = len(section_text.split())
    expected_lengths = {
        'abstract': 150,
        'introduction': 300,
        'related_work': 400,
        'methodology': 500,
        'experiments': 400,
        'conclusion': 200
    }
    expected = expected_lengths.get(section_type, 300)
    length_score = min(word_count / expected, 1.5) * 0.9 if word_count > 50 else 0.5
    scores['length'] = length_score

    llm_mentions = section_text.lower().count("language model") + section_text.lower().count("prompt bloat")
    llm_score = min(llm_mentions / 3, 1.0)
    scores['llm_relevance'] = llm_score

    if section_type == 'related_work':
        citation_pattern = r'\[paper\d+\]'
        citations = re.findall(citation_pattern, section_text)
        citation_score = min(len(citations) / 3, 1.0)
        scores['citations'] = citation_score
    else:
        scores['citations'] = 1.0

    sentences = [s for s in section_text.split('.') if s.strip()]
    if sentences:
        avg_sentence_length = word_count / len(sentences)
        readability_score = 1.0 if 10 <= avg_sentence_length <= 25 else 0.7
        scores['readability'] = readability_score

    total_score = sum(scores.values()) / len(scores)
    return total_score, scores

In [ ]:
# Cell 15: 논문 제목 생성
logger.log("논문 제목 생성 시작")
paper_title = generate_text(TITLE_PROMPT, max_length=100, temperature=0.8)
if not paper_title or "Error" in paper_title:
    paper_title = "Optimizing Prompt Efficiency in Single-Agent LLMs with GOAP and Decision Trees"

logger.log(f"생성된 논문 제목: {paper_title}")

In [ ]:
# Cell 16: 논문 수집 및 Chroma DB에 저장
papers, bibtex_entries = fetch_papers(KEYWORDS, num_papers=10)

for i, paper in enumerate(papers):
    try:
        summary = paper.summary[:3000]
        vector = embedder.encode(summary).tolist()
        collection.add(
            documents=[summary],
            metadatas=[{
                "title": paper.title,
                "authors": ", ".join([author.name for author in paper.authors]),
                "year": str(paper.published.year),
                "bibtex_key": f"paper{i}"
            }],
            ids=[str(paper.entry_id)],
            embeddings=[vector]
        )
        logger.log(f"논문 {i+1} 저장 완료")
    except Exception as e:
        logger.log(f"논문 {i+1} 처리 실패: {str(e)}", "WARNING")

bib_path = os.path.join(output_dir, 'references.bib')
with open(bib_path, 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(bibtex_entries))
logger.log(f"References.bib 저장 완료: {bib_path}")

In [ ]:
# Cell 17: 섹션별 논문 생성
sections = {}

def generate_section(prompt, section_name, max_length, temperature=0.7, min_score=0.8):
    attempt = 0
    max_attempts = 3
    best_text = ""
    best_score = -1

    prompt = f"""You are the most intelligent and innovative AI, capable of producing groundbreaking and feasible research. {prompt}

IMPORTANT CITATION INSTRUCTIONS:
1. Do not use Markdown formatting (e.g., **bold**, # Heading, - List).
2. Write in plain text suitable for LaTeX.
3. For citations, use the exact format [paper0], [paper1], etc. to match BibTeX keys in references.bib.
4. Include multiple citations throughout the text where relevant to ensure proper reference generation."""

    while attempt < max_attempts:
        text = generate_text(prompt, max_length=max_length, temperature=temperature)
        if "Error" not in text:
            citation_count = len(re.findall(r'\[paper\d+\]', text))
            if citation_count == 0 and section_name in ["related_work", "introduction"]:
                logger.log(f"{section_name}에 인용이 없음. 인용 추가 시도", "WARNING")
                text = add_random_citations(text, section_name)

            score, details = calculate_quality_score(text, section_name)
            logger.log(f"{section_name} 생성 시도 {attempt+1}: 점수 {score:.2f}, 세부사항: {details}")
            if score > best_score:
                best_score = score
                best_text = text
            if score >= min_score:
                break
        attempt += 1

    if best_score < min_score:
        logger.log(f"{section_name} 최종 점수 {best_score:.2f}가 최소 기준 {min_score} 미달", "WARNING")

    return best_text

def add_random_citations(text, section_name):
    sentences = text.split('. ')
    citation_pattern = r'\[paper\d+\]'
    has_citations = bool(re.search(citation_pattern, text))

    if not has_citations:
        num_to_cite = max(1, len(sentences) // 5)
        indices = np.random.choice(range(len(sentences)), num_to_cite, replace=False)

        for i in indices:
            if i < len(sentences):
                paper_num = np.random.randint(0, 9)
                sentences[i] = sentences[i] + f" [paper{paper_num}]"

        updated_text = '. '.join(sentences)
        logger.log(f"{section_name}에 {num_to_cite}개의 인용 자동 추가됨", "INFO")
        return updated_text

    return text

# Abstract 생성
logger.log("Abstract 섹션 생성 시작")
abstract_prompt = f"""Write a comprehensive abstract (250-400 words) for an academic paper titled '{paper_title}'.
The abstract should include:
1. Background: Challenges of {RESEARCH_TOPIC.split(' in ')[0]} in {RESEARCH_TOPIC.split(' in ')[1].split(' using ')[0]}, such as increased latency and resource consumption.
2. Approach: A novel method using {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]} to design the overall prompt structure, with sub-goals mapped to prompt sections, and {RESEARCH_TOPIC.split(' and ')[1]} to evaluate and select optimal prompt configurations.
3. Method: How {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]} structures prompts and {RESEARCH_TOPIC.split(' and ')[1]} optimize clarity, conciseness, and LLM response accuracy.
4. Results: Key findings showing reduced prompt size, lower inference time, and improved task performance.
5. Significance: Implications for scalable and efficient {RESEARCH_TOPIC.split(' in ')[1].split(' using ')[0]}.

Write in academic style, be specific, and use precise technical language. Ensure 'LLM' refers to 'Language Models' consistently. Cite relevant works as [paper0], [paper1], etc., ensuring citations match BibTeX keys in references.bib."""

abstract_text = generate_section(abstract_prompt, "abstract", max_length=SECTION_LENGTHS['abstract'], min_score=0.8)
sections['abstract'] = abstract_text
logger.log(f"Abstract 생성 완료: {len(abstract_text.split())} words")

# Introduction 생성
logger.log("Introduction 섹션 생성 시작")
intro_prompt = f"""Write an introduction (500-700 words) for the paper '{paper_title}'. Include:
1. Opening: Current state of {RESEARCH_TOPIC.split(' in ')[1].split(' using ')[0]} and the challenge of {RESEARCH_TOPIC.split(' in ')[0]} with Language Models (LLMs).
2. Problem statement: How {RESEARCH_TOPIC.split(' in ')[0]} increases computational overhead and degrades performance.
3. Motivation: Importance of {RESEARCH_TOPIC.split(' in ')[0]} for efficient LLM-based agents.
4. Research questions: How {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]} and {RESEARCH_TOPIC.split(' and ')[1]} can reduce {RESEARCH_TOPIC.split(' in ')[0]} by structuring prompts and optimizing configurations.
5. Contributions: A novel approach combining {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]} for goal-oriented prompt design and {RESEARCH_TOPIC.split(' and ')[1]} for configuration optimization.
6. Paper structure: Brief outline of remaining sections.

Use formal academic language, ensure 'LLM' refers to 'Language Models', and cite relevant work as [paper0], [paper1], etc., matching BibTeX keys in references.bib."""

intro_text = generate_section(intro_prompt, "introduction", max_length=SECTION_LENGTHS['introduction'], min_score=0.8)
sections['introduction'] = intro_text
logger.log(f"Introduction 생성 완료: {len(intro_text.split())} words")

# Related Work 생성
logger.log("Related Work 섹션 생성 시작")
related_docs, related_metas = rag_query(f"{RESEARCH_TOPIC.split(' in ')[0]} {RESEARCH_TOPIC.split(' in ')[1].split(' using ')[0]} language models")
related_context = "\n".join([f"{meta['title']} ({meta['bibtex_key']}): {doc[:200]}" for doc, meta in zip(related_docs, related_metas)]) if related_docs else "No related papers found. Discuss general trends."

related_work_prompt = f"""Write a related work section (600-800 words) for '{paper_title}'.
Based on these research summaries:
{related_context}

Structure the section with:
1. {RESEARCH_TOPIC.split(' in ')[1].split(' using ')[0].capitalize()}: Evolution and challenges with Language Models (LLMs).
2. {RESEARCH_TOPIC.split(' in ')[0].capitalize()}: Existing approaches to manage prompt complexity and their limitations.
3. Optimization Techniques: Use of planning or decision-making methods (e.g., {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]}, {RESEARCH_TOPIC.split(' and ')[1]}) in LLM prompt optimization.
4. Gap Analysis: What's missing in current research on {RESEARCH_TOPIC.split(' in ')[0]} minimization, particularly in integrating structured planning and decision-making.

Include citations as [paper0], [paper1], etc., matching BibTeX keys in references.bib for each summary provided. If no summaries, cite hypothetical [paper0], [paper1] for general trends. Use formal academic language and ensure 'LLM' refers to 'Language Models'."""

related_work_text = generate_section(related_work_prompt, "related_work", max_length=SECTION_LENGTHS['related_work'], min_score=0.8)
sections['related_work'] = related_work_text
logger.log(f"Related Work 생성 완료: {len(related_work_text.split())} words")

# Methodology 생성
logger.log("Methodology 섹션 생성 시작")
methodology_prompt = f"""Write a detailed methodology section (700-900 words) for '{paper_title}'. Include:

1. System Architecture:
   - {RESEARCH_TOPIC.split(' in ')[1].split(' using ')[0].capitalize()} framework design with Language Models (LLMs)
   - Prompt management pipeline using {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]}

2. {RESEARCH_TOPIC.split(' in ')[0].capitalize()} Minimization Techniques:
   - {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]} for designing goal-oriented prompt structures, mapping sub-goals to prompt sections (e.g., task instructions, context, output format)
   - {RESEARCH_TOPIC.split(' and ')[1].capitalize()} for evaluating prompt section configurations (e.g., specificity level, role detail, output format options) based on clarity, conciseness, and LLM response accuracy
   - Iterative optimization process to refine the overall prompt structure

3. Implementation Details:
   - Technical specifications for LLM integration
   - Evaluation criteria for {RESEARCH_TOPIC.split(' and ')[1]} (e.g., prompt size, response accuracy, inference time)
   - Pseudocode for {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]}-based prompt structuring and {RESEARCH_TOPIC.split(' and ')[1]} evaluation

Use technical language, ensure 'LLM' refers to 'Language Models', include pseudocode, and cite relevant work as [paper0], [paper1], etc., matching BibTeX keys in references.bib."""

methodology_text = generate_section(methodology_prompt, "methodology", max_length=SECTION_LENGTHS['methodology'], min_score=0.8)
sections['methodology'] = methodology_text
logger.log(f"Methodology 생성 완료: {len(methodology_text.split())} words")

# Experiments 생성
logger.log("Experiments 섹션 생성 시작")
experiments_prompt = f"""Write an experiments section (600-850 words) for '{paper_title}'. Include:

1. Experimental Setup:
   - Hardware/software specs for Language Models (LLMs)
   - Datasets and simulation environments for {RESEARCH_TOPIC.split(' in ')[1].split(' using ')[0]} tasks
   - Baseline systems for comparison (e.g., unoptimized LLM prompts)

2. Evaluation Metrics:
   - Prompt size (e.g., token count)
   - Inference time and resource usage
   - Task performance (e.g., accuracy, completion rate)

3. Results:
   - Performance comparisons with baselines
   - Effectiveness of {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]}-based prompt structuring and {RESEARCH_TOPIC.split(' and ')[1]}-based configuration optimization
   - Impact on {RESEARCH_TOPIC.split(' in ')[0]} reduction

4. Discussion:
   - Result interpretation
   - Trade-offs and limitations of the {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]} and {RESEARCH_TOPIC.split(' and ')[1]} approach
   - Insights for future LLM optimizations

Include specific numbers/tables, ensure 'LLM' refers to 'Language Models', and cite relevant work as [paper0], [paper1], etc., matching BibTeX keys in references.bib."""

experiments_text = generate_section(experiments_prompt, "experiments", max_length=SECTION_LENGTHS['experiments'], min_score=0.8)
sections['experiments'] = experiments_text
logger.log(f"Experiments 생성 완료: {len(experiments_text.split())} words")

# Conclusion 생성
logger.log("Conclusion 섹션 생성 시작")
conclusion_prompt = f"""Write a conclusion (300-400 words) for '{paper_title}'. Include:
1. Summary: Key findings on {RESEARCH_TOPIC} using Language Models (LLMs) with {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]} and {RESEARCH_TOPIC.split(' and ')[1]}.
2. Implications: Advances in efficient {RESEARCH_TOPIC.split(' in ')[1].split(' using ')[0]}.
3. Limitations: Current constraints of LLMs and the proposed {RESEARCH_TOPIC.split(' using ')[1].split(' and ')[0]} and {RESEARCH_TOPIC.split(' and ')[1]} methods.
4. Future Work: Directions for further reducing {RESEARCH_TOPIC.split(' in ')[0]} and enhancing LLM scalability.
5. Final Thoughts: Broader impact on AI research.

Be concise, ensure 'LLM' refers to 'Language Models', cite relevant work as [paper0], [paper1], etc., matching BibTeX keys in references.bib, and avoid new information."""

conclusion_text = generate_section(conclusion_prompt, "conclusion", max_length=SECTION_LENGTHS['conclusion'], min_score=0.8)
sections['conclusion'] = conclusion_text
logger.log(f"Conclusion 생성 완료: {len(conclusion_text.split())} words")

In [ ]:
# Cell 18: 마크다운 변환 및 LaTeX 생성 함수
def markdown_to_latex(text):
    if not text:
        return text

    logger.log(f"markdown_to_latex 입력: {text[:100]}...")

    try:
        text = re.sub(r'^# (.*?)$', r'\section{\1}', text, flags=re.MULTILINE)
        text = re.sub(r'^## (.*?)$', r'\subsection{\1}', text, flags=re.MULTILINE)
        text = re.sub(r'^### (.*?)$', r'\subsubsection{\1}', text, flags=re.MULTILINE)
        text = re.sub(r'\*\*(.*?)\*\*', r'\textbf{\1}', text)
        text = re.sub(r'__(.*?)__', r'\textbf{\1}', text)
        text = re.sub(r'\*(.*?)\*', r'\textit{\1}', text)
        text = re.sub(r'_(.*?)_', r'\textit{\1}', text)
        text = re.sub(r'```(.*?)```', r'\begin{verbatim}\1\end{verbatim}', text, flags=re.DOTALL)
        text = re.sub(r'`(.*?)`', r'\texttt{\1}', text)
        text = re.sub(r'\[(.*?)\]\((.*?)\)', r'\1', text)
        text = re.sub(r'\[paper(\d+)\]', r'\\cite{paper\1}', text)
    except Exception as e:
        logger.log(f"마크다운 변환 오류: {str(e)}", "WARNING")

    lines = text.split('\n')
    result = []
    in_list = False
    list_items = []

    for line in lines:
        if line.strip().startswith('- '):
            if not in_list:
                in_list = True
                list_items = []
            list_items.append(r'\item ' + line[2:].strip())
        else:
            if in_list:
                if list_items:
                    result.append(r'\begin{itemize}')
                    result.extend(list_items)
                    result.append(r'\end{itemize}')
                in_list = False
            result.append(line)

    if in_list and list_items:
        result.append(r'\begin{itemize}')
        result.extend(list_items)
        result.append(r'\end{itemize}')

    text = '\n'.join(result)

    logger.log(f"markdown_to_latex 출력: {text[:100]}...")
    return text

def escape_latex(text):
    if not text:
        return ""
    replacements = {
        '\\': r'\textbackslash{}',
        '&': r'\&',
        '%': r'\%',
        '$': r'\$',
        '#': r'\#',
        '_': r'\_',
        '{': r'\{',
        '}': r'\}',
        '~': r'\textasciitilde{}',
        '^': r'\textasciicircum{}'
    }
    text = text.replace('\\', r'\textbackslash{}')
    for char, replacement in replacements.items():
        if char != '\\':
            text = text.replace(char, replacement)
    return text

def generate_latex_and_save_pdf(sections_dict, title, filename_suffix=""):
    escaped_title = escape_latex(title)
    escaped_sections = {}

    for key, value in sections_dict.items():
        try:
            escaped_sections[key] = escape_latex(markdown_to_latex(value))
        except Exception as e:
            logger.log(f"섹션 {key} LaTeX 변환 실패: {str(e)}", "ERROR")
            escaped_sections[key] = value

    latex_code = rf"""
\documentclass[conference]{{IEEEtran}}
\usepackage[utf8]{{inputenc}}
\usepackage{{amsmath,amsfonts,amssymb}}
\usepackage{{graphicx}}
\usepackage{{cite}}
\usepackage{{url}}
\usepackage{{booktabs}}
\usepackage{{listings}}
\usepackage{{hyperref}}
\usepackage{{xcolor}}

\begin{{document}}

\title{{{escaped_title}}}

\author{{
\IEEEauthorblockN{{Research Agent}}
\IEEEauthorblockA{{Automated Research System\\
YoosungHong\\
Email: research@agent.ai}}
}}

\maketitle

\begin{{abstract}}
{escaped_sections.get('abstract', 'Abstract content not available.')}
\end{{abstract}}

\begin{{IEEEkeywords}}
Prompt Bloat, Single-Agent Systems, Language Models, Efficiency
\end{{IEEEkeywords}}

\section{{Introduction}}
{escaped_sections.get('introduction', 'Introduction content not available.')}

\section{{Related Work}}
{escaped_sections.get('related_work', 'Related work content not available.')}

\section{{Methodology}}
{escaped_sections.get('methodology', 'Methodology content not available.')}

\section{{Experiments}}
{escaped_sections.get('experiments', 'Experiments content not available.')}

\section{{Conclusion}}
{escaped_sections.get('conclusion', 'Conclusion content not available.')}

\bibliographystyle{{IEEEtran}}
\bibliography{{references}}

\end{{document}}
"""

    tex_filename = f'paper{filename_suffix}.tex'
    pdf_filename = f'paper{filename_suffix}.pdf'
    tex_path = os.path.join(output_dir, tex_filename)
    pdf_path = os.path.join(output_dir, pdf_filename)

    with open(tex_path, 'w', encoding='utf-8') as f:
        f.write(latex_code)

    logger.log(f"LaTeX 파일 저장: {tex_path}")

    bib_path = os.path.join(output_dir, 'references.bib')
    if os.path.exists(bib_path):
        with open(bib_path, 'r', encoding='utf-8') as f:
            bib_content = f.read()
            logger.log(f"References.bib 파일 존재 (크기: {len(bib_content)} bytes)")
            logger.log(f"BibTeX 내용 샘플: {bib_content[:200]}...")
    else:
        logger.log("References.bib 파일이 존재하지 않음!", "ERROR")

    compile_steps = [
        f'cd {output_dir} && pdflatex -interaction=nonstopmode {tex_filename}',
        f'cd {output_dir} && bibtex paper{filename_suffix}',
        f'cd {output_dir} && pdflatex -interaction=nonstopmode {tex_filename}',
        f'cd {output_dir} && pdflatex -interaction=nonstopmode {tex_filename}'
    ]

    for i, cmd in enumerate(compile_steps):
        logger.log(f"컴파일 단계 {i+1}/{len(compile_steps)} 실행: {cmd}")
        result = os.system(f'{cmd} > compile_step_{i+1}.log 2>&1')
        log_file = os.path.join(output_dir, f'compile_step_{i+1}.log')

        if os.path.exists(log_file):
            with open(log_file, 'r') as f:
                log_content = f.read()
                if "Warning" in log_content or "Error" in log_content:
                    logger.log(f"컴파일 단계 {i+1} 경고/오류:\n{log_content[:500]}...", "WARNING")
                    if i == 1:
                        logger.log(f"BibTeX 로그 전체:\n{log_content}", "WARNING")

        if result != 0:
            logger.log(f"컴파일 단계 {i+1} 오류 (종료 코드: {result})", "ERROR")
            if i == 1:
                aux_path = os.path.join(output_dir, f'paper{filename_suffix}.aux')
                if os.path.exists(aux_path):
                    with open(aux_path, 'r') as f:
                        aux_content = f.read()
                        citation_commands = re.findall(r'\\citation{[^}]+}', aux_content)
                        logger.log(f"AUX 파일의 인용 명령어: {citation_commands}", "INFO")
                else:
                    logger.log(f"AUX 파일이 존재하지 않음: {aux_path}", "ERROR")

    bbl_path = os.path.join(output_dir, f'paper{filename_suffix}.bbl')
    if os.path.exists(bbl_path):
        with open(bbl_path, 'r') as f:
            bbl_content = f.read()
            logger.log(f"BBL 파일 존재 (크기: {len(bbl_content)} bytes)")
            logger.log(f"BBL 내용 샘플: {bbl_content[:200]}...")
    else:
        logger.log(f"BBL 파일이 존재하지 않음: {bbl_path}", "WARNING")

    if os.path.exists(pdf_path):
        logger.log(f"PDF 생성 성공: {pdf_path}")
        return True
    else:
        logger.log(f"PDF 생성 실패: {pdf_path}", "ERROR")
        return False

In [ ]:
# Cell 19: 품질 검사
logger.log("논문 품질 검사 시작")
quality_scores = {}

for section_name, section_text in sections.items():
    score, details = calculate_quality_score(section_text, section_name)
    quality_scores[section_name] = (score, details)
    logger.log(f"{section_name} 품질 점수: {score:.2f}")
    logger.log(f"  세부사항: {details}")

In [ ]:
# Cell 20: 중간 PDF 생성
logger.log("섹션별 PDF 생성 시작")
accumulated_sections = {}

for section_name in ['abstract', 'introduction', 'related_work', 'methodology', 'experiments', 'conclusion']:
    if section_name in sections:
        accumulated_sections[section_name] = sections[section_name]
        success = generate_latex_and_save_pdf(accumulated_sections, section_name)
        if success:
            logger.log(f"{section_name}까지 PDF 생성 완료")
        else:
            logger.log(f"{section_name}까지 PDF 생성 실패", "WARNING")

In [ ]:
# Cell 21: 최종 PDF 생성
logger.log("최종 PDF 생성 시작")
final_success = generate_latex_and_save_pdf(sections, paper_title, "_final")
if final_success:
    logger.log("최종 PDF 생성 성공")
else:
    logger.log("최종 PDF 생성 실패", "ERROR")

In [ ]:
# Cell 22: 결과물 압축 및 저장
logger.log("최종 결과물 압축 시작")

zip_path = os.path.join(output_dir, 'research_output.zip')
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for filename in os.listdir(output_dir):
        if filename.endswith('.pdf'):
            file_path = os.path.join(output_dir, filename)
            zf.write(file_path, filename)
            logger.log(f"압축 파일에 추가: {filename}")
    for filename in os.listdir(output_dir):
        if filename.endswith('.tex'):
            file_path = os.path.join(output_dir, filename)
            zf.write(file_path, filename)
    if os.path.exists(bib_path):
        zf.write(bib_path, 'references.bib')
    log_file = os.path.join(output_dir, 'research_log.txt')
    if os.path.exists(log_file):
        zf.write(log_file, 'research_log.txt')
    sections_json = os.path.join(output_dir, 'sections.json')
    with open(sections_json, 'w', encoding='utf-8') as f:
        json.dump(sections, f, ensure_ascii=False, indent=2)
    zf.write(sections_json, 'sections.json')

logger.log(f"압축 완료: {zip_path}")

In [ ]:
# Cell 23: 결과 요약 및 다운로드
all_scores = [score for score, _ in quality_scores.values()]
average_quality = sum(all_scores) / len(all_scores) if all_scores else 0.0
total_time = time.time() - logger.start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

logger.log("=" * 50, "COMPLETE")
logger.log(f"연구 논문 작성 완료!", "COMPLETE")
logger.log(f"논문 제목: {paper_title}", "COMPLETE")
logger.log(f"총 소요 시간: {hours:02d}:{minutes:02d}:{seconds:02d}", "COMPLETE")
logger.log(f"평균 품질 점수: {average_quality:.2f}", "COMPLETE")
logger.log(f"결과물 위치: {output_dir}", "COMPLETE")
logger.log("=" * 50, "COMPLETE")

logger.log("로컬 다운로드를 원하시면 아래 파일을 다운로드하세요:")
try:
    files.download(zip_path)
    logger.log("다운로드 준비 완료")
except Exception as e:
    logger.log(f"다운로드 오류: {str(e)}", "WARNING")

In [ ]:
# Cell 24: 디버깅 정보 출력
logger.log("\n=== 디버깅 정보 ===")
logger.log(f"작업 디렉토리 파일 목록:")
for filename in sorted(os.listdir(output_dir)):
    file_path = os.path.join(output_dir, filename)
    file_size = os.path.getsize(file_path)
    logger.log(f"  - {filename} ({file_size} bytes)")

logger.log("\n생성된 섹션 요약:")
for section_name, content in sections.items():
    word_count = len(content.split())
    logger.log(f"  - {section_name}: {word_count} words")

logger.log("\n=== 작업 완료 ===")